Find cross section of actors and movies?


In [2]:
// TODO: use papa parse to load tsv files from imdb
var util = require('util');
var fs = require('fs');
var papa = require('papaparse');
var sqlite3 = require('better-sqlite3');

var principals = '/Users/briancullinan/Downloads/title.principals.tsv';
var movies = './movies.sqlite.db';

var file = fs.createReadStream(principals, { highWaterMark: 1024 * 1024 });
var db = new sqlite3.Database(movies, sqlite3.OPEN_READWRITE | sqlite3.OPEN_CREATE);
var runAsync = util.promisify(db.run.bind(db));

function importSQL() {
    var prev = 0;
    var count = 0;
    return new Promise(resolve => papa.parse(file, {
        worker: true,
        delimiter: '\t',
        fastMode: true,
        step: function(results) {
            count++;
            var sql = `
INSERT OR IGNORE INTO principals
    (tconst, ordering, nconst, category, job, characters)
VALUES (?,?,?,?,?,?)`
            if(count > prev + 1000) {
                prev = count;
                console.log(`${count} - ${results.data}`);
            }
            return runAsync(sql, results.data)
        },
        complete: resolve
    }));
}

function createTables() {
    return runAsync(`
CREATE TABLE IF NOT EXISTS principals (
    tconst TEXT NOT NULL,
    ordering INT NOT NULL,
    nconst TEXT NOT NULL,
    category TEXT,
    job TEXT,
    characters TEXT,
    UNIQUE(tconst,ordering),
    PRIMARY KEY (tconst, ordering)
);
`)
        .then(() => runAsync(`
CREATE INDEX IF NOT EXISTS principles_titles
ON principals (tconst);`))
        .then(() => runAsync(`
CREATE INDEX IF NOT EXISTS principles_names
ON principals (nconst);`))
}

$$.async();
createTables()
    .then(() => importSQL())
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
    // close the database connection
    .then(() => db.close())



/*
params = [[1,2],[3,4],[5,6],[7,8]]

stmt = db.prepare("INSERT INTO test (num1, num2) VALUES (?1,?2)");
stmt.all([params], function(err, res) {
    ...
});
*/



Error: The module '/Users/briancullinan/jupytangular2/node_modules/better-sqlite3/build/better_sqlite3.node'
was compiled against a different Node.js version using
NODE_MODULE_VERSION 67. This version of Node.js requires
NODE_MODULE_VERSION 64. Please try re-compiling or re-installing
the module (for instance, using `npm rebuild` or `npm install`).